#### Steps

- Import the CSV file into a pandas df
- Clean the table name and remove all extra sysmbols, spaces, capital letter
- Clean the colum headers and remove all extra symbols, spaces and capital letter
- Write the create tbale SQL statement
- Import the data into the db

In [12]:
# find excel files in my current working directory
# isolate only the excel files
# make a new directory
# move the excel files in the new directory

In [54]:
# importing libraries
# !pip install psycopg2

import os
import numpy as np
import pandas as pd
import psycopg2

#Import the CSV file into a pandas df
df = pd.read_excel('Corruption Perception Index Data Set.xlsx', sheet_name="CPI Timeseries 2012 - 2020")

# Promote the second row to headers
new_headers = df.iloc[1]
df.columns = new_headers

# Drop the first two rows
df = df.iloc[2:].reset_index(drop=True)

columns_to_drop = [col for col in df.columns if col.startswith('Standard') or col.startswith('Sources')]
df = df.drop(columns=columns_to_drop)

file = 'Corruption Perception Index Data Sets'
clean_tbl_name = file.lower().replace(" ", "_")

#Clean the colum headers and remove all extra symbols, spaces and capital letter
df.columns = [x.lower().replace(" ", "_") for x in df.columns]

df.fillna(0, inplace=True)

df = df.apply(lambda x: x.astype(int) if x.name not in ['country', 'iso3', 'region'] else x)



replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int32' : 'int',
    'datetime64' : 'timestamp'
}

col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))

# df.head()
# col_str

# conn_string = 'host="localhost", port="5432", dbname="WDI", user="postgres", password="admin"' 
conn = psycopg2.connect(host="localhost", port="5432", dbname="WDI", user="postgres", password="admin")
cursor = conn.cursor()
# print('Opened databse successfully')

# drop table with same name
cursor.execute("drop table if exists corruption_perception_index_data_sets;")
# create table 
cursor.execute("create table corruption_perception_index_data_sets(%s)" %(col_str))
# print('success')


# save df to csv
df.to_csv('corruption_perception_index_data_sets.csv', header=df.columns, index=False, encoding='utf-8')

#open the csv file

my_file = open('corruption_perception_index_data_sets.csv')
# print('file opened in memory')

SQL_STATEMENT = """
COPY corruption_perception_index_data_sets FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
# print('file copied to db')

cursor.execute("grant select on corruption_perception_index_data_sets to public")
conn.commit()
cursor.close()
print('table corruption_perception_index_data_sets import to db completed')

table corruption_perception_index_data_sets import to db completed
